In [6]:
import pandas as pd
import numpy as np

funding = pd.read_csv('/Users/duncanwan/Desktop/learning/Bitcoin/Clean_data/btcfunding_agg.csv',
                     usecols = ['calc_time', 'last_funding_rate'])

spot = pd.read_csv('/Users/duncanwan/Desktop/learning/Bitcoin/4hrs/spot/cleaned/BTCUSDT-4h-2025-04-08-cleaned.csv',
                  usecols = ['close_time','close','volume'])

prep = pd.read_csv('/Users/duncanwan/Desktop/learning/Bitcoin/4hrs/future/cleaned/BTCUSDT-4h-2025-04-08-cleaned.csv',
                  usecols = ['close_time','close','volume'])

# <span style="background-color: #ff0000">Changed: Use floor() instead of ceil() to round DOWN</span>
spot["close_time"] = pd.to_datetime(spot["close_time"], utc=True).dt.round('s')  # <span style="background-color: #ff0000">CHANGED: ceil → floor</span>
prep["close_time"] = pd.to_datetime(prep["close_time"], utc=True).dt.round('s')  # <span style="background-color: #ff0000">CHANGED: ceil → floor</span>
funding["calc_time"] = pd.to_datetime(funding["calc_time"], utc=True).dt.round('s')  # <span style="background-color: #ff0000">REMOVED .dt.ceil() - keep original 8h timestamps</span>

spot["close_time"] = spot["close_time"].dt.round('H')
prep["close_time"] = prep["close_time"].dt.round('H')
funding["calc_time"] = funding["calc_time"].dt.round('H')


spot = spot.rename(columns={"close_time":"bar_end",
                           "close":"spot_close",
                           "volume":"spot_vol"})
prep = prep.rename(columns={"close_time":"bar_end",
                           "close":"perp_close",
                           "volume":"perp_vol"})
funding= funding.rename(columns={"last_funding_rate":"funding"})

spot = spot.sort_values("bar_end").drop_duplicates("bar_end")
prep = prep.sort_values("bar_end").drop_duplicates("bar_end")
funding = funding.sort_values("calc_time").drop_duplicates("calc_time")

# Merge spot and prep
bars = (
    spot[['bar_end','spot_close','spot_vol']]
    .merge(prep[['bar_end','perp_close','perp_vol']], on='bar_end', how='inner')
    .sort_values('bar_end')
)

# <span style="background-color: #ff0000">CRITICAL FIX: Changed direction from 'backward' to 'forward'</span>
f8 = funding[['calc_time','funding']].sort_values('calc_time')

bars = pd.merge_asof(
    bars,
    f8,
    left_on='bar_end',
    right_on='calc_time',
    direction='backward',  # <span style="background-color: #ff0000">CHANGED: backward → forward</span>
    allow_exact_matches=True
).drop(columns='calc_time').rename(columns={'funding':'funding_rate'})

bars.to_csv('/Users/duncanwan/Desktop/learning/Bitcoin/4hrs/BTC_combined_2504_08v4.csv')

/var/folders/kz/cfb6kmcd0j5g728cbyjlwwvw0000gp/T/ipykernel_59172/184857900.py:18: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  spot["close_time"] = spot["close_time"].dt.round('H')
/var/folders/kz/cfb6kmcd0j5g728cbyjlwwvw0000gp/T/ipykernel_59172/184857900.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  prep["close_time"] = prep["close_time"].dt.round('H')
/var/folders/kz/cfb6kmcd0j5g728cbyjlwwvw0000gp/T/ipykernel_59172/184857900.py:20: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  funding["calc_time"] = funding["calc_time"].dt.round('H')
